In [1]:
import json
import os
import re
import numpy as np
import pandas as pd
from scipy import sparse
from scipy.stats import uniform
from tqdm import tqdm

In [2]:
song_meta = pd.read_json('3rd-melon-baseline/res/song_meta.json',encoding='utf-8') #곡 정보가 들어있는 데이터. song id, artist id, genre 등의 정보 있다.
train_data = pd.read_json('3rd-melon-baseline/res/train.json',encoding='utf-8') #예비테스트용 데이터. playlist id, 그 playlist 당 song id리스트들&태그& 등등이 있다. (장르x. song은 리스트로 표현되어있음)
val_data = pd.read_json('3rd-melon-baseline/res/val.json',encoding='utf-8') #예비테스트용 데이터. playlist id, 그 playlist 당 song id리스트들&태그& 등등이 있다. (장르x. song은 리스트로 표현되어있음)

In [3]:
id_songs = (
    train_data[['id', 'songs']]
    .explode('songs')
    .assign(value=1)
    .rename(columns={'id':'user_id', 'songs':'item_id'})
)

In [4]:
# range of int32
assert id_songs['user_id'].max() < 2147483647
assert id_songs['item_id'].max() < 2147483647

In [5]:
id_songs['user_id'] = id_songs['user_id'].astype(np.int32)
id_songs['item_id'] = id_songs['item_id'].astype(np.int32)
id_songs['value'] = id_songs['value'].astype(np.int8)

In [6]:
id_songs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3943148 entries, 0 to 82574
Data columns (total 3 columns):
user_id    int32
item_id    int32
value      int8
dtypes: int32(2), int8(1)
memory usage: 63.9 MB


In [7]:
while True:
    prev = len(id_songs)

    # 5곡 이상 가진 플레이 리스트만
    user_count = id_songs.user_id.value_counts()
    id_songs = id_songs[id_songs.user_id.isin(user_count[user_count >= 5].index)]

    # 5번 이상 등장한 곡들만
    item_count = id_songs.item_id.value_counts()
    id_songs = id_songs[id_songs.item_id.isin(item_count[item_count >= 5].index)]

    cur = len(id_songs)

    if prev==cur: break
    
    print("제거 데이터 수: ", prev-cur)

제거 데이터 수:  710870
제거 데이터 수:  14244
제거 데이터 수:  1868
제거 데이터 수:  410
제거 데이터 수:  194
제거 데이터 수:  91
제거 데이터 수:  56
제거 데이터 수:  72
제거 데이터 수:  84
제거 데이터 수:  9


참조: https://stackoverflow.com/questions/31661604/efficiently-create-sparse-pivot-tables-in-pandas

In [8]:
from scipy.sparse import csr_matrix
from pandas.api.types import CategoricalDtype

user_cate = CategoricalDtype(sorted(id_songs.user_id.unique()), ordered=True)
song_cate = CategoricalDtype(sorted(id_songs.item_id.unique()), ordered=True)

song_row = id_songs.user_id.astype(user_cate).cat.codes
song_col = id_songs.item_id.astype(song_cate).cat.codes
song_sparse_matrix = csr_matrix((id_songs["value"], (song_row, song_col)), \
                           shape=(user_cate.categories.size, song_cate.categories.size))


In [9]:
song_idx2id = {i:j for i, j in enumerate(song_cate.categories)}
song_id2idx = {j:i for i, j in enumerate(song_cate.categories)}

참조: https://implicit.readthedocs.io/en/latest/quickstart.html

In [10]:
import implicit

In [11]:
# initialize a model
model_song = implicit.als.AlternatingLeastSquares(factors=50)

# train the model on a sparse matrix of item/user/confidence weights
model_song.fit(song_sparse_matrix.T)

# recommend items for a user
user_items = song_sparse_matrix.tocsr()

In [12]:
# 인기곡들 보자
top_songs = id_songs.item_id.value_counts().nlargest(10)
top_songs

30314784    1590
3620493     1494
2960594     1428
8235260     1357
623902      1209
3568916     1197
2939095     1127
4543502     1099
30492279    1063
4369827     1056
Name: item_id, dtype: int64

In [13]:
target_song = top_songs.index[0]
related = model_song.similar_items(song_id2idx[target_song])
related = [song_idx2id[r[0]] for r in related]

In [14]:
song_meta[song_meta._id==target_song] # 비슷한 걸 잘 추천해주는 거 같다

,song_gn_dtl_gnr_basket,issue_date,_id,album_id,artist_id_basket,song_name,song_gn_gnr_basket,album_name,artist_name_basket
359662,"[GN0105, GN0101]",20170324,30314784,10047890,261143,밤편지,[GN0100],밤편지,아이유


In [15]:
song_meta[song_meta._id.isin(related)]

,song_gn_dtl_gnr_basket,issue_date,_id,album_id,artist_id_basket,song_name,song_gn_gnr_basket,album_name,artist_name_basket
248306,"[GN0105, GN0101]",20161008,30028465,10004630,718655,이별 여행,[GN0100],불후의 명곡 - 전설을 노래하다 (작곡가 신재홍 편),이예준
359662,"[GN0105, GN0101]",20170324,30314784,10047890,261143,밤편지,[GN0100],밤편지,아이유
371455,"[GN0401, GN0403]",20170407,30349593,10052968,261143,사랑이 잘 (With 오혁),[GN0400],사랑이 잘,아이유
381660,"[GN0401, GN0403]",20170421,30378156,10056666,261143,팔레트 (Feat. G-DRAGON),[GN0400],Palette,아이유
444380,[GN0101],20170805,30561898,10085011,1514,저녁 바다,[GN0100],소길9화,장필순
477803,"[GN0805, GN0801]",20170922,30636089,10096855,261143,가을 아침,[GN0800],꽃갈피 둘,아이유
486495,"[GN0105, GN2505, GN2501, GN0101, GN2503]",20170928,30655509,10099857,896283 1628740,시월에 설악산,"[GN2500, GN0100]",Autumn,예빈 (다이아) 솜이 (다이아)
500752,"[GN0303, GN0301]",20171023,30688500,10105030,108356,연애소설 (Feat. 아이유),[GN0300],WE`VE DONE SOMETHING WONDERFUL,에픽하이 (EPIK HIGH)
771925,"[GN0105, GN0101]",20170421,30378157,10056666,261143,이런 엔딩,[GN0100],Palette,아이유
826931,[GN1701],20160126,8030177,2663871,7267,Broken Wing,[GN1700],Chet Baker Live In Tokyo (Memorial Box) Part.2,Chet Baker


In [16]:
# 정합성 볼 때 체크
# display(song_meta[song_meta._id.isin(related)])

In [17]:
# 미리 탑 200 땡겨놓기
top200_songs = id_songs.groupby('item_id').value.sum().nlargest(200).index

In [18]:
def song_recommender(ipt):
    '''
    인풋을 하나밖에 못 받아서 ipt로 받은 다음 두 개로 unpacking
    여러 개 받는 방법 있을 거 같은데 모름 ...
    top200_songs를 따로 input으로 받은 이유는, 분산처리에서 여러 코어가 하나의 객체를 접근할 때 오류가 나기 때문
    그래서 밑에 top200_song을 input으로 줄 때 굳이 copy 해서 줬다.
    근데 근데 joblib에서는 났었는데, multiprocessing에서는 안난다 ;; 두 개의 성질이 좀 다른 것 같다
    즉 이렇게 안해줘도 에러는 안나는데 .... 안전빵으로 해줫음
    '''
    row, top200_songs = ipt
    cands = {}
    # 각 곡 별로
    for song in row.songs:
        # 유사곡 100개 소환. 
        if song not in song_id2idx.keys(): continue # val에는 train에 아예 없던 곡이 나올 수 있으므로 해당하면 재끼도록 설계
        related = [(song_idx2id[r[0]], r[1]) for r in model_song.similar_items(song_id2idx[song], 100)]
        for cand in related:
            # 추천된 아이템 : 유사도 리스트를 딕셔너리로 구현
            cands[cand[0]] = cands.get(cand[0], []) + [cand[1]]
    # 한 곡이 여러번 추천될 땐 가장 높은 유사도 하나 채택
    cands = {k:max(v) for k, v in cands.items()}
    # 유사도 순으로 정렬 후, 기존 플레이리스트에 없는 곳들만 가지고 100개 뽑기
    sorted_cands = [w for w in sorted(cands, key=cands.get, reverse=True) if w not in row.songs][:100]

    # 가끔 미쳐가지고 비어있거나 한 경우도 있음. 이럴 땐 그냥 베스트를 넣어주자
    if len(sorted_cands) < 100:
        non_seen_top_200_songs = [song for song in top200_songs if song not in row.songs + sorted_cands]
        sorted_cands += non_seen_top_200_songs[:100-len(sorted_cands)]
    
    assert len(sorted_cands) == 100
    
    # float가 많아서 int로 전환
    sorted_cands = [int(s) for s in sorted_cands]
#     song_rec[row.id] = sorted_cands
    return (row.id, sorted_cands)

In [19]:
from multiprocessing import Pool

In [20]:
%%time
with Pool(8) as p:
    res = p.map(song_recommender, [(row, top200_songs.copy()) for idx, row in tqdm(val_data.iterrows(), total=len(val_data))])    

100%|██████████| 22521/22521 [00:02<00:00, 10212.94it/s]


CPU times: user 7.07 s, sys: 775 ms, total: 7.84 s
Wall time: 5min 37s


In [21]:
# 각 플레이리스트에 대해
song_rec = {k:v for (k, v) in res}

## TAG 작업

In [22]:
id_tags = (
    train_data[['id', 'tags']]
    .explode('tags')
    .assign(value=1)
    .rename(columns={'id':'user_id', 'tags':'tag_name'})
)

In [23]:
# range of int32
assert id_tags['user_id'].max() < 2147483647

In [24]:
id_tags['user_id'] = id_tags['user_id'].astype(np.int32)
id_tags['value'] = id_tags['value'].astype(np.int8)

In [25]:
id_tags.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 336840 entries, 0 to 82574
Data columns (total 3 columns):
user_id     336840 non-null int32
tag_name    336840 non-null object
value       336840 non-null int8
dtypes: int32(1), int8(1), object(1)
memory usage: 6.7+ MB


In [26]:
while True:
    prev = len(id_tags)

    # 5곡 이상 가진 플레이 리스트만
    user_count = id_tags.user_id.value_counts()
    id_tags = id_tags[id_tags.user_id.isin(user_count[user_count >= 5].index)]

    # 5번 이상 등장한 곡들만
    tag_count = id_tags.tag_name.value_counts()
    id_tags = id_tags[id_tags.tag_name.isin(tag_count[tag_count >= 5].index)]

    cur = len(id_tags)

    if prev==cur: break
    
    print("제거 데이터 수: ", prev-cur)

제거 데이터 수:  138904
제거 데이터 수:  13269
제거 데이터 수:  1499
제거 데이터 수:  275
제거 데이터 수:  63
제거 데이터 수:  16
제거 데이터 수:  20
제거 데이터 수:  12


In [27]:
user_cate = CategoricalDtype(sorted(id_tags.user_id.unique()), ordered=True)
tag_cate = CategoricalDtype(sorted(id_tags.tag_name.unique()), ordered=True)

tag_row = id_tags.user_id.astype(user_cate).cat.codes
tag_col = id_tags.tag_name.astype(tag_cate).cat.codes
tag_sparse_matrix = csr_matrix((id_tags["value"], (tag_row, tag_col)), \
                           shape=(user_cate.categories.size, tag_cate.categories.size))


In [28]:
tag_idx2name = {i:j for i, j in enumerate(tag_cate.categories)}
tag_name2idx = {j:i for i, j in enumerate(tag_cate.categories)}

In [29]:
# initialize a model
model_tag = implicit.als.AlternatingLeastSquares(factors=50)

# train the model on a sparse matrix of item/user/confidence weights
model_tag.fit(tag_sparse_matrix.T)

# recommend items for a user
user_tags = tag_sparse_matrix.tocsr()

In [30]:
# 인기태그들 보자
top_tags = id_tags.tag_name.value_counts().nlargest(10)
top_tags

감성      6342
기분전환    6069
드라이브    4691
카페      4089
잔잔한     3812
휴식      3586
사랑      3039
발라드     2848
힐링      2745
신나는     2663
Name: tag_name, dtype: int64

In [31]:
# 1위인 감성과 관련된 태그들
related = model_tag.similar_items(tag_name2idx[top_tags.index[0]])
[tag_idx2name[r[0]] for r in related]

['감성', '스푼자몽', '스푼', '자몽', '갬성', '가사가좋은노래', '밤에', '퇴근_후', '헤이즈', '음색좋은']

In [32]:
# 미리 탑 200 땡겨놓기
top200_tags = id_tags.groupby('tag_name').value.sum().nlargest(200).index

In [34]:
# 각 플레이리스트에 대해
tag_rec = {}
for idx, row in tqdm(val_data.iterrows(), total=len(val_data)):
    cands = {}
    # 각 태그 별로
    for tag in row.tags:
        # 유사곡 100개 소환. 
        if tag not in tag_name2idx.keys(): continue # val에는 train에 아예 없던 곡이 나올 수 있으므로 해당하면 재끼도록 설계
        related = [(tag_idx2name[r[0]], r[1]) for r in model_tag.similar_items(tag_name2idx[tag], 10)]
        for cand in related:
            # 추천된 아이템 : 유사도 리스트를 딕셔너리로 구현
            cands[cand[0]] = cands.get(cand[0], []) + [cand[1]]
    # 한 곡이 여러번 추천될 땐 가장 높은 유사도 하나 채택
    cands = {k:max(v) for k, v in cands.items()}
    # 유사도 순으로 정렬 후, 기존 플레이리스트에 없는 곳들만 가지고 100개 뽑기
    sorted_cands = [w for w in sorted(cands, key=cands.get, reverse=True) if w not in row.tags][:10]

    # 가끔 미쳐가지고 비어있거나 한 경우도 있음. 이럴 땐 그냥 베스트를 넣어주자
    if len(sorted_cands) < 10:
        non_seen_top_200_tags = [tag for tag in top200_tags if tag not in row.tags + sorted_cands]
        sorted_cands += non_seen_top_200_tags[:10-len(sorted_cands)]
    
    assert len(sorted_cands) == 10

    tag_rec[row.id] = sorted_cands

100%|██████████| 22521/22521 [00:57<00:00, 390.75it/s]


### 잘 됐나 확인

In [35]:
target = val_data.sample()
print(target.songs)
print(target.tags)

15648    [3026155, 3049084, 3136470, 5704784, 1889399, ...
Name: songs, dtype: object
15648    []
Name: tags, dtype: object


In [36]:
tag_rec[target['id'].values[0]] # 비슷한 tag가 보인다

['감성', '기분전환', '드라이브', '카페', '잔잔한', '휴식', '사랑', '발라드', '힐링', '신나는']

In [37]:
song_meta.loc[song_meta._id.isin(target.songs.values[0]), ['song_name', 'artist_name_basket']]

,song_name,artist_name_basket
19649,너에게..기대,메이트
206046,Gavial,국카스텐
296519,봄이 오는 동안,재주소년
321659,겨울밤,장재인
349522,별 빛이 내린다,안녕바다
389725,그게 아니고,10CM
578550,놀이 (Feat. San E),양다일 강민희
582401,우리는 선처럼 가만히 누워 (Feat. 이상순),요조
708139,북치는 토끼,루싸이트 토끼
738266,River,어반자카파


In [38]:
song_meta.loc[song_meta._id.isin(song_rec[target['id'].values[0]]), ['song_name', 'artist_name_basket']]

,song_name,artist_name_basket
19186,돌아오면 돼,박지윤
19662,안녕,메이트
19736,왜,메이트
26139,멀어지네요,심현보
39887,거꾸로 걷는다,어반자카파
...,...,...
811322,난시,딕펑스 (DICKPUNKS)
817335,그대는 어디에 (Feat. 한희정),에피톤 프로젝트
820211,이별 통보 (Vocal 반광옥),더필름
825893,괜찮아졌어 (Feat. 김지영),Ignite


In [39]:
target = val_data.sample()
print(target.songs)
print(target.tags)

19499    []
Name: songs, dtype: object
19499    []
Name: tags, dtype: object


In [40]:
tag_rec[target['id'].values[0]] # 주어진 tag가 없다보니 global top을 갔다

['감성', '기분전환', '드라이브', '카페', '잔잔한', '휴식', '사랑', '발라드', '힐링', '신나는']

In [41]:
song_meta.loc[song_meta._id.isin(target.songs.values[0]), ['song_name', 'artist_name_basket']]

,song_name,artist_name_basket


In [42]:
# 중복 없나 확인
for k, v in tqdm(song_rec.items()):
    if len(v) != len(set(v)):
        print(k)

100%|██████████| 22521/22521 [00:00<00:00, 222653.65it/s]
